# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
# Contigency table of call results and their probabilities by race 
cross_tab = pd.crosstab(data.race == 'b', data.call == 1)
cross_tab.columns = ['No_call', 'Called']
cross_tab.index = ['White', 'Black']
cross_tab['prob'] = cross_tab.Called / (cross_tab.No_call + cross_tab.Called)
cross_tab

,No_call,Called,prob
White,2200,235,0.096509
Black,2278,157,0.064476


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### What test is appropriate for this problem? Does CLT apply?
As we want to know if there is any difference in call results by race, we split data into two subser for whites and blacks, repectively. Then we compute the proportion of sucess on call and implement hypothesis test using bootstrap replicates and frequentist method.

From the contegency table above, the number of observations of four cells are all greater than ten (or five), so we can apply CLT to use the normal apploximation to binomial. 

### What are the null and alternate hypotheses?
Lets p1 be the probability of receiving a call for whites and p2 for blacks.

H0: p1 - p2 = 0 (no difference between whites and blacks)

H1: p1 - p2 > 0 (whites have a higher probability than blacks)

In [6]:
# Split dataset
w = data[data.race=='w']
b = data[data.race=='b']

### Compute margin of error, confidence interval, and p-value. 
### Try using both the bootstrapping and the frequentist statistical approaches.

In [7]:
# Bootstrap 
def frac_diff(data1, data2): # function for difference of fractions
    frac1 = np.sum(data1) / len(data1)
    frac2 = np.sum(data2) / len(data2)
    return frac1 - frac2

def draw_bs_reps(data1, data2, func, size = 1): # function for bootstrap replicates
    bs_reps = np.empty(size)
    
    for i in range(size):
        bs_sample1 = np.random.choice(data1, len(data1))
        bs_sample2 = np.random.choice(data2, len(data2))
        bs_reps[i] = func(bs_sample1, bs_sample2)
    
    return bs_reps

# fraction difference between whites and black
obs_frac_diff = frac_diff(w.call, b.call)

# fraction differences of bootstrap samples
bs_frac_diff_reps = draw_bs_reps(w.call, b.call, frac_diff, size = 10000) 

# 95% confidence interval, margin of error and p-value
conf_int_95 = np.percentile(bs_frac_diff_reps, [2.5, 97.5])
margin_of_error = (conf_int_95[1] - conf_int_95[0]) /2
p = np.sum(bs_frac_diff_reps < 0) / len(bs_frac_diff_reps)

# print outputs
print('Fraction difference of observation: ', obs_frac_diff,
      '\nMargin of error: ', margin_of_error,
      '\n95% confidence interval: ', conf_int_95,
      '\np-value of fraction difference from bootstrap: ', p)

Fraction difference of observation:  0.032032854209445585 
Margin of error:  0.015400410677618066 
95% confidence interval:  [0.0164271  0.04722793] 
p-value of fraction difference from bootstrap:  0.0


In [8]:
# Chi-squre test
# Recall the contingency table from eralir work
observed = cross_tab.Called 
w_frac = cross_tab.prob[0]
b_frac = cross_tab.prob[1]
diff_frac = w_frac - b_frac
exp_frac = np.sum(data.call) / len(data.call)
expected = [exp_frac * len(w.call), exp_frac * len(b.call)]
expected

# Compute Chi-sq, margin of error, 95% confidence interval and p-value
chi_sq = ((observed - expected) ** 2 / expected).sum().sum()
se_diff = np.sqrt(np.sum([w_frac * (1 - w_frac) / len(w.call), b_frac * (1 - b_frac) / len(b.call)]))
margin_of_error = stats.norm.ppf(.975) * se_diff
conf_int_95 = [diff_frac - margin_of_error, diff_frac + margin_of_error]
p = 1 - stats.chi2.cdf(chi_sq, df = 1)

# Print outputs
print('Fraction difference of observation: ', obs_frac_diff,
      '\nMargin of error: ', margin_of_error,
      '\n95% confidence interval: ', conf_int_95,
      '\np-value of fraction difference from Chi-square test: ', p)

Fraction difference of observation:  0.032032854209445585 
Margin of error:  0.01525512602821483 
95% confidence interval:  [0.016777728181230755, 0.04728798023766041] 
p-value of fraction difference from Chi-square test:  8.161930359706915e-05


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

### Write a story describing the statistical significance in the context or the original problem.
We performed two different hypothsis tests: one is an empirical method using bootsrtap resampling and the other one is a frequentist method using Chi-square test.  The results are almost same in confidence interval of (0.017, 0.047). Though the p-value of bootstrap test is a bit larger than the other one, both of the values are small enough to agree with that the two groups of the race - with white-sounding and black-sounding names - have not the same probabilities to receive a callback after applying a job.

### Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
We cannot say the race is the most important factor in callback success.  This is because there are over 60 other characteristics to make a decision on the call.  For example, the education level and experience would be different between those two groups, and these might be the primary predictors to the callback success.  To find the effectness, we should consider a model that consists of all relavent variables to the call and analyse it.